# DS-GA 1019

# Lab 9: Concurrency 
## Mar. 30, 2023

In [1]:
import time
import threading

def do_something():
    print('Sleeping 1 second...')
    time.sleep(1)    # wait
    print('Done sleeping')

In [2]:
start = time.time()

do_something()
do_something()

finish = time.time()

print(f'Finished in {(finish - start):.1f} seconds')

Sleeping 1 second...
Done sleeping
Sleeping 1 second...
Done sleeping
Finished in 2.0 seconds


#### Sequential code

<img   src="images/im1.png" alt="Drawing" style="width: 500px;"/>


#### Multi threaded code

<img   src="images/im2.png" alt="Drawing" style="width: 500px;"/>


- only do one func at one time due to python GIL
- but will be faster because when interpreter is resting in func 1, it may work on func2

Process vs. threads:
- threads share the same memory
- process: each has it's own memory; run on different CPU

In [3]:
start = time.time()
# Thread is the class that create thread:
t1 = threading.Thread(target = do_something)  # for no arg func
t2 = threading.Thread(target = do_something)
# > create threads
t1.start() # start threads
t2.start()

#wait for the threads to finish:
# need to use .join() when you want to combine the outputs 
t1.join()  # to make sure the func finish
t2.join()  # because

finish = time.time()

print(f'Finished in {(finish - start):.1f} seconds')

Sleeping 1 second...
Sleeping 1 second...
Done sleepingDone sleeping

Finished in 1.0 seconds


In [4]:
def do_something_specific(seconds):
    print(f'Sleeping {seconds} seconds...')
    time.sleep(seconds)
    print('Done sleeping')

In [5]:
start = time.time()

threads = []

for _ in range(10):  # create 10 threads
    t = threading.Thread(target = do_something_specific, args = [2]) # pass a list of arg
    t.start()
    threads.append(t)

for thread in threads:
    thread.join()

finish = time.time()

print(f'Finished in {(finish - start):.1f} seconds')

Sleeping 2 seconds...
Sleeping 2 seconds...
Sleeping 2 seconds...
Sleeping 2 seconds...
Sleeping 2 seconds...
Sleeping 2 seconds...
Sleeping 2 seconds...
Sleeping 2 seconds...
Sleeping 2 seconds...
Sleeping 2 seconds...
Done sleepingDone sleeping

Done sleeping
Done sleeping
Done sleeping
Done sleeping
Done sleeping
Done sleeping
Done sleeping
Done sleeping
Finished in 2.0 seconds


### Queue

In [6]:
from queue import Queue 

my_queue = Queue(maxsize=0) #FIFO queue
my_queue.put(1) # add something into a queue
my_queue.put(2)
my_queue.put(3)
print(my_queue.get()) # get things from queue


1


In [7]:
q = Queue()
num_threads = 2

In [8]:
def worker():
    while True:
        print(f'Waiting for message, id = {threading.get_ident()}')
        item = q.get()  # if didn't get anythin -> stop
        print(f'Message received = {item}, id = {threading.get_ident()}')
        
        if item is not None:
            do_work(item)
            q.task_done()  
        else:
            q.task_done()
            break

In [9]:
def do_work(item):
    print(f'Processing message .... {threading.get_ident()} -- {item}')
    time.sleep(2)
    print(f'Message processed .... {threading.get_ident()} -- {item}')

In [10]:
threads = []

for _ in range(num_threads):
    t = threading.Thread(target = worker)
    t.start()
    threads.append(t)

Waiting for message, id = 13139910656Waiting for message, id = 13156700160



In [11]:
#Add items to queue
for item in ['wuphf','dot','com']:
    q.put(item)

Message received = wuphf, id = 13156700160Message received = dot, id = 13139910656
Processing message .... 13139910656 -- dot

Processing message .... 13156700160 -- wuphf


In [12]:
#Print all running threads
threading.enumerate()

[<_MainThread(MainThread, started 8643868160)>,
 <Thread(IOPub, started daemon 13020753920)>,
 <Heartbeat(Heartbeat, started daemon 13037543424)>,
 <Thread(Thread-3, started daemon 13055406080)>,
 <Thread(Thread-4, started daemon 13072195584)>,
 <ControlThread(Control, started daemon 13088985088)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 13105774592)>,
 <ParentPollerUnix(Thread-2, started daemon 13123121152)>,
 <Thread(Thread-17, started 13139910656)>,
 <Thread(Thread-18, started 13156700160)>]

In [ ]:
#stop threads
for _ in range(num_threads):
    q.put(None)

### Multiprocessing

Tasks are executed on multiple processors / cpus

In [1]:
!pip install multiprocess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 8.6 MB/s eta 0:00:00
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)


In [2]:
import multiprocess as mp
print("Number of processors: ", mp.cpu_count())

Number of processors:  8


### Example: Calculate e^x of elements of an array

In [3]:
import numpy as np
arr = np.random.randint(0, 10, size=[5000])
data = arr.tolist()


In [4]:
def factorial_upto(n):
    res = [1]
    
    for i in range(1, n + 1):
        res.append(res[-1] * i)
    return res

def taylor_exp(x,n=1000):
    
    factorials = factorial_upto(n)
    
    res = 0
    for i in range(n):
        res += x**i / factorials[i]
    
    return res

### Sequential solution

In [7]:
start = time.time()
results = []
for x in data:
    results.append(taylor_exp(x))

print(f'Finished in {(time.time() - start):.4f} seconds')


Finished in 7.4004 seconds


### Parallelizing using Pool.map
- Pool() start the processing
- pool.map(function, data)


In [8]:
import time
start = time.time()

pool = mp.Pool(mp.cpu_count())

results_mp = pool.map(my_func, [x for x in data])

pool.close()
print(f'Finished in {(time.time() - start):.4f} seconds')

assert (results_mp == results)

Finished in 1.1421 seconds


In [ ]:
a = 5
b = a
b += 3
a,b

### Parallelizing using Pool.starmap

Lets us pass multiple arguments

In [ ]:
start = time.time()

pool = mp.Pool(mp.cpu_count())

results_smp = pool.starmap(taylor_exp, [(x,1000) for x in data])

pool.close()
print(f'Finished in {(time.time() - start):.4f} seconds')

assert (results_smp == results)

In [ ]:
!python3 pool_star.py

### Global interpreter lock

Only one thread can access the interpreter at a time due to GIL

Python releases GIL 

- while a thread is waiting for IO
- while numpy is doing an array operation: because numpy runs C in backend. 

More info on Python GIL - _https://realpython.com/python-gil/_

In [ ]:
import math

def f(x): #Doesnot releast GIL
    print (x)
    y = [1]*5000000
    [math.exp(i) for i in y]
    
def g(x):   #Releases GIL
    print (x)
    y = np.ones(5000000)
    np.exp(y)

def do_work(q,func):
    while True:
        item = q.get()
        
        if item is not None:
            func(item)
            q.task_done()
        else:
            q.task_done()
            break

### serial f()

In [ ]:
start = time.time()

for i in range(10):
    f(i)

print(f'Finished in {(time.time() - start):.4f} seconds')


### threaded f()

In [ ]:
from queue import Queue
start = time.time()

q = Queue()
num_threads = 4

for i in range(num_threads):
    worker = threading.Thread(target = do_work, args = (q,f)) # refer to q
    worker.setDaemon(True) # this stop the threads when the program quits  
    # > like with open(), will automatically stops
    worker.start()         # start the threads

# now we have started 10 threads:

for i in range(10):
    q.put(i)

q.join()
print(f'Finished in {(time.time() - start):.4f} seconds')


### parallel f()

In [ ]:
start = time.time()

pool = mp.Pool(4)

results_mp = pool.map(f, [x for x in range(10)])

pool.close()
print(f'Finished in {(time.time() - start):.4f} seconds')


In [ ]:
!python3 parallel_f.py

### serial g()

In [ ]:
start = time.time()

for i in range(100):
    g(i)

print(f'Finished in {(time.time() - start):.4f} seconds')


### threaded g()

In [ ]:
start = time.time()

q = Queue()
num_threads = 4

for i in range(num_threads):
    worker = threading.Thread(target = do_work, args = (q,g)) # refer to q
    worker.setDaemon(True) # this stop the threads when the program quits  
    worker.start()         # start the threads

# now we have started 10 threads:

for i in range(100):
    q.put(i)

q.join()
print(f'Finished in {(time.time() - start):.4f} seconds')
